In [1]:
import os
import time
import json
import pandas as pd
import torch
import loader
import utils
import preprocess_data
from config import config_base
from config import config_r_net
from config import config_match_lstm
from config import config_bi_daf
from modules import match_lstm
from modules import r_net
from modules import bi_daf

# config
config = config_match_lstm.config

In [2]:
# load vocab
lang = loader.load_vocab(config.vocab_path)

In [3]:
embedding_np = loader.load_w2v(config.embedding_path)

In [4]:
test_data = loader.load_data(config.test_df, lang)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.452 seconds.
Prefix dict has been built succesfully.


In [5]:
test_loader = loader.build_loader(
        dataset=test_data[:2],
        batch_size=config.batch_size,
        shuffle=False,
        drop_last=False
    )

In [6]:
param = {
        'embedding': embedding_np,
        'embedding_type': config.embedding_type,
        'embedding_is_training': config.embedding_is_training,
        'mode': config.mode,
        'hidden_size': config.hidden_size,
        'dropout_p': config.dropout_p,
        'encoder_dropout_p': config.encoder_dropout_p,
        'encoder_bidirectional': config.encoder_bidirectional,
        'encoder_layer_num': config.encoder_layer_num,
        'is_bn': config.is_bn
    }

In [7]:
model = eval(config.model_name).Model(param)
model = model.cuda()

In [8]:
    model_path = os.path.join('model', config.model_save)
    print('load model, ', model_path)
    state = torch.load(model_path)
    model.load_state_dict(state['best_model_state'])

    best_loss = state['best_loss']
    best_epoch = state['best_epoch']
    best_step = state['best_step']
    time_use = state['time']
    print('best_epoch:%2d, best_step:%5d, best_loss:%.4f, best_time:%d' % (best_epoch, best_step, best_loss, time_use))

load model,  model/match_lstm_1
best_epoch: 9, best_step:24700, best_loss:1.6659, best_time:31607


In [9]:
for batch in test_loader:
    batch = batch
    break

In [10]:
batch = utils.deal_batch(batch)

In [11]:
outputs = model(batch)

In [12]:
outputs.size()

torch.Size([2, 32, 293])

In [13]:
answer_prop = outputs
mask = utils.get_mask(batch[0])
max_tokens = 50

In [17]:
a, b = answer_search(answer_prop, mask, max_tokens)

In [23]:
aa = a.cpu().numpy()
aa

array([ 24, 112,  37,   9,  47,  82, 105,  40,  50,  15,  83,  89,  66,
        11, 136,  37,   4,  82,  53,  77,  27,  69,  44,  56, 152,  44,
       108,  35,  93,  19,  69,  43])

In [27]:
ans_s_p.data.cpu().numpy()

array([[2.8570977e-01, 1.5757346e-03, 1.3563248e-04, ..., 1.0000000e-06,
        1.0000000e-06, 1.0000000e-06],
       [2.2726918e-03, 7.3753281e-06, 1.0440446e-04, ..., 1.0000000e-06,
        1.0000000e-06, 1.0000000e-06],
       [1.8052991e-06, 8.6687521e-07, 5.0409599e-06, ..., 1.0000000e-06,
        1.0000000e-06, 1.0000000e-06],
       ...,
       [5.2722113e-04, 6.1415209e-05, 1.1539657e-04, ..., 1.0000000e-06,
        1.0000000e-06, 1.0000000e-06],
       [4.2834989e-07, 2.0494163e-07, 4.3651485e-06, ..., 1.0000000e-06,
        1.0000000e-06, 1.0000000e-06],
       [3.7689028e-07, 2.5841240e-07, 5.0667222e-08, ..., 1.0000000e-06,
        1.0000000e-06, 1.0000000e-06]], dtype=float32)

In [16]:
answer_search(answer_prop, mask, max_tokens)

(tensor([  24,  112,   37,    9,   47,   82,  105,   40,   50,   15,
           83,   89,   66,   11,  136,   37,    4,   82,   53,   77,
           27,   69,   44,   56,  152,   44,  108,   35,   93,   19,
           69,   43], device='cuda:0'),
 tensor([  29,  112,   38,   11,  195,   87,  105,   40,   53,   15,
           86,   90,   70,   14,  176,   40,    8,   92,   54,  100,
           30,   79,   45,   60,  171,   47,  110,   35,  106,   21,
           79,   46], device='cuda:0'))

In [21]:
ans_s_p = answer_prop[0]
ans_e_p = answer_prop[1]

In [15]:
def answer_search(answer_prop, mask, max_tokens):
    """
     global search best answer for model predict
    :param answer_prop: (2, batch_size, c_len)
    :param mask: (batch_size, c_len)
    :param max_tokens: .
    :return: ans_range, score
    """
    batch_size = answer_prop.size(1)
    c_len = answer_prop.size(2)

    # get min length
    lengths = mask.data.eq(1).long().sum(dim=1).squeeze()
    min_length, _ = torch.min(lengths, 0)
    min_length = min_length.item()

    # max move steps
    max_move = max_tokens + c_len - min_length
    max_move = min(c_len, max_move)

    ans_s_p = answer_prop[0]
    ans_e_p = answer_prop[1]
    b_zero = answer_prop.new_zeros(batch_size, 1)

    ans_s_e_p_lst = []
    for i in range(max_move):
        temp_ans_s_e_p = ans_s_p * ans_e_p
        ans_s_e_p_lst.append(temp_ans_s_e_p)

        ans_s_p = ans_s_p[:, :(c_len - 1)]
        ans_s_p = torch.cat([b_zero, ans_s_p], dim=1)

    ans_s_e_p = torch.stack(ans_s_e_p_lst, dim=2)

    # get the best end position, and move steps
    max_prop1, max_prop_idx1 = torch.max(ans_s_e_p, 1)
    max_prop2, max_prop_idx2 = torch.max(max_prop1, 1)

    ans_e = max_prop_idx1.gather(1, max_prop_idx2.unsqueeze(1)).squeeze(1)
    ans_s = ans_e - max_prop_idx2

    return ans_s, ans_e